In [1]:
!pip install torchtext==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.5 MB/s eta 0:00:00


## Download Dataset

In [2]:
# https://drive.google.com/file/d/1bdTvpHW-cfAMpM3YqcZI_d2ANX8BE6g0/view?usp=sharing
!gdown --id 1bdTvpHW-cfAMpM3YqcZI_d2ANX8BE6g0

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1bdTvpHW-cfAMpM3YqcZI_d2ANX8BE6g0
To: /content/poems.csv
100% 71.3k/71.3k [00:00<00:00, 64.8MB/s]


## Import Library

In [3]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from collections import Counter

In [4]:
df = pd.read_csv('poems.csv')
df

,Unnamed: 0,title,content
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC..."
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...
...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh..."
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...


In [5]:
df = df.drop(columns=['Unnamed: 0'])
df

,title,content
0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...
1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...
2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...
3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC..."
4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...
...,...,...
185,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh..."
186,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...
187,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...
188,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...


In [6]:
df['content'][0].split('\n')

['Cái làm ta hạnh phúc',
 'Thực ra cũng chẳng nhiều',
 'Chỉ cần có ai đó',
 'Để ta thầm thương yêu',
 '',
 'Rồi thêm chút công việc',
 'Cho ta làm hàng ngày',
 'Cuối cùng, chút mơ mộng',
 'Để đưa ta lên mây']

## Text Normalize And Tokenize

In [7]:
def text_normalize(text):
    return text.strip()

df['content'] = df['content'].apply(lambda x: text_normalize(x))

def tokenize(text):
    return text.split()

## Build Vocab

In [8]:
counter = Counter()

specials = ['<unk>', '<pad>', '<sos>', '<eos>', '<eol>']

for text in df['content']:
    counter.update(tokenize(text))

for special in specials:
    counter.update(tokenize(special))

vocab = Vocab(
    counter,
    specials=specials,
    min_freq=1,
)

class VocabWrapper:
    def __init__(self, vocab):
        self.vocab = vocab
        self.unk_index = vocab['<unk>']

    def __call__(self, text):
        return self.vocab.stoi.get(text, self.unk_index)

In [9]:
vocab_wrapper = VocabWrapper(vocab)
PAD_TOKEN = vocab['<pad>']
EOS_TOKEN = vocab['<eos>']

MAX_SEQ_LEN = 25

def pad_and_truncate(sequence, max_seq_len):
    if len(sequence) > max_seq_len:
        return sequence[:max_seq_len]
    else:
        pad = [PAD_TOKEN] * (max_seq_len - len(sequence))
        return sequence + pad

def vectorize(text, max_seq_len=MAX_SEQ_LEN):
    sequence = [vocab_wrapper(token) for token in tokenize(text)]
    sequence = pad_and_truncate(sequence, max_seq_len)
    return sequence

def decode(sequence):
    return [vocab.itos[token] for token in sequence]


In [10]:
print(df['content'][0].split('\n')[0])
print(vectorize(df['content'][0].split('\n')[0], 10))

Cái làm ta hạnh phúc
[175, 62, 39, 313, 366, 1, 1, 1, 1, 1]


## Create Dataset

In [11]:
class PoemDataset(Dataset):
    def __init__(
        self,
        df,
        tokenizer,
        vectorizer,
        max_seq_len=MAX_SEQ_LEN
    ):
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.padding_masks = self.create_sample(df)

    def create_padding_mask(self, input_seq, pad_token_id = PAD_TOKEN):
        return [1 if token != pad_token_id else 0 for token in input_seq]

    def split_content(self, content):
        samples = []
        poem_parts = content.split('\n\n')

        for poem_part in poem_parts:
            lines = poem_part.split('\n')
            samples.append(lines)

        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        padding_masks = []

        input_text = '<sos> ' + ' <eol> '.join(sample) + ' <eol> <eos>'
        input_text = self.tokenizer(input_text)

        for idx in range(1, len(input_text)):
            input_seq = ' '.join(input_text[:idx])
            target_seq = ' '.join(input_text[1:idx+1])
            input_seq = self.vectorizer(input_seq, self.max_seq_len)
            target_seq = self.vectorizer(target_seq, self.max_seq_len)
            padding_mask = self.create_padding_mask(input_seq)

            input_seqs.append(input_seq)
            target_seqs.append(target_seq)
            padding_masks.append(padding_mask)

        return input_seqs, target_seqs, padding_masks

    def create_sample(self, df):
        input_seqs = []
        target_seqs = []
        padding_masks = []
        for content in df['content']:
            samples = self.split_content(content)
            for sample in samples:
                input_seq, target_seq, padding_mask = self.prepare_sample(sample)
                input_seqs.extend(input_seq)
                target_seqs.extend(target_seq)
                padding_masks.extend(padding_mask)

        input_seqs = torch.tensor(input_seqs, dtype=torch.long)
        target_seqs = torch.tensor(target_seqs, dtype=torch.long)
        padding_masks = torch.tensor(padding_masks, dtype=torch.float)

        return input_seqs, target_seqs, padding_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        input_seqs = self.input_seqs[idx]
        target_seqs = self.target_seqs[idx]
        padding_masks = self.padding_masks[idx]

        return input_seqs, target_seqs, padding_masks

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
train_batch_size = 256
train_dataset = PoemDataset(
    df,
    tokenizer=tokenize,
    vectorizer=vectorize,
    max_seq_len=MAX_SEQ_LEN
)

train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=False,
)

In [14]:
input_seqs, target_seqs, padding_masks = next(iter(train_loader))
print(input_seqs[0])
print(target_seqs[0])
print(padding_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
tensor([175,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


## Positional Encoding

In [15]:
class PositionalEncoding(nn.Module):
    def __init__(
        self,
        empedding_dims,
        dropout,
        max_len=5000
    ):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, empedding_dims, 2) * -(math.log(10000.0) / empedding_dims))

        pe = torch.zeros(max_len, 1, empedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## Transformer Model

In [16]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embending_dims,
        num_heads,
        hidden_dims,
        num_layers,
        dropout = 0.5,
    ):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embending_dims)
        self.embedding_dims = embending_dims

        self.pos_encoder = PositionalEncoding(embending_dims, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embending_dims,
            nhead=num_heads,
            dim_feedforward=hidden_dims,
            dropout=dropout,
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layers,
            num_layers=num_layers,
        )
        self.decoder = nn.Linear(embending_dims, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None,padding_masks=None):
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)
        if src_mask is not None:
            src_mask = nn.Transformer.generate_square_subsequent_mask(src.size(0)).to(src.device)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=padding_masks)
        output = self.decoder(output)
        return output

## Init Parameter

In [17]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIIDEN_DIMS = 128
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2

model = TransformerModel(
    vocab_size=VOCAB_SIZE,
    embending_dims=EMBEDDING_DIMS,
    num_heads=N_HEADS,
    hidden_dims=HIIDEN_DIMS,
    num_layers=N_LAYERS,
    dropout=DROPOUT,
).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [18]:
input_tests = torch.randint(0, 10, (10, 10)).to(device)
with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

torch.Size([10, 10, 2201])


In [19]:
lr = 5.0
n_epochs = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

## Train Model

In [20]:
model.train()
for epoch in range(n_epochs):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, padding_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        padding_masks = padding_masks.to(device).permute(1, 0)

        output = model(input_seqs, padding_masks=padding_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'Epoch: {epoch}, Loss: {total_loss}')
    scheduler.step()

Epoch: 0, Loss: 4.796652576753071
Epoch: 1, Loss: 3.7344376274517606
Epoch: 2, Loss: 3.5621146644864763
Epoch: 3, Loss: 3.561149375779288
Epoch: 4, Loss: 3.389056384563446
Epoch: 5, Loss: 3.143863631146295
Epoch: 6, Loss: 2.6764938959053586
Epoch: 7, Loss: 2.150296758328165
Epoch: 8, Loss: 1.7491020389965601
Epoch: 9, Loss: 1.4468609648091453
Epoch: 10, Loss: 1.2245536680732454
Epoch: 11, Loss: 1.0570845635873931
Epoch: 12, Loss: 0.9242939533931869
Epoch: 13, Loss: 0.825789346226624
Epoch: 14, Loss: 0.7674616692321641
Epoch: 15, Loss: 0.7137822966490474
Epoch: 16, Loss: 0.6861673950084618
Epoch: 17, Loss: 0.6665448961513383
Epoch: 18, Loss: 0.6406860431390149
Epoch: 19, Loss: 0.6150846646300384
Epoch: 20, Loss: 0.6024964946721282
Epoch: 21, Loss: 0.5765304842165538
Epoch: 22, Loss: 0.5710250101983547
Epoch: 23, Loss: 0.5626940046037946
Epoch: 24, Loss: 0.5470812368605819
Epoch: 25, Loss: 0.5286148254360471
Epoch: 26, Loss: 0.5251496785453388
Epoch: 27, Loss: 0.5165211686066219
Epoch: 2

In [21]:
def sample_with_temperature(logits, temperature = 1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)
    samples = torch.multinomial(probabilities, 1).item()
    return samples

In [36]:
model.eval()
temperature = 1.9
input_text = '<sos> Nhớ'
input_tokens = tokenize(input_text)
input_ids = [vocab_wrapper(token) for token in input_tokens]
eos_token_id = vocab['<eos>']
generated_ids = input_ids.copy()
MAX_GENEARTION_LEN = 50
for _ in range(MAX_GENEARTION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        output = model(input_tensor)
        logits = output[0, -1, :]
        sample_id = sample_with_temperature(logits, temperature)
        generated_ids.append(sample_id)
        if sample_id == eos_token_id:
            break

generated_text = decode(generated_ids)
generated_text = ' '.join(generated_text)
generated_text = generated_text.replace('<sos>', '').replace('<eos>', '')
lines = generated_text.split('<eol>')
for line in lines:
    print(''.join(line))

 Nhớ dạo 
 Nằm cây Thời gian như bất tận 
 Em giận 
 Khi Nào ít Giọng của ven hổ 
 Cặn quá nứt cả Chuyện được quá vợ bán bất đều nồng trăng thổi vật 
 dù 
 Áo giác đáp nói 
 Tấm sức! 
 cho sự không


In [27]:
torch.save(model.state_dict(), 'poem_generation_model.pth')